In [1]:
from sqlalchemy import create_engine, inspect
import mlflow
import pandas as pd
import numpy as np
import os

In [2]:
from sklearn.model_selection import train_test_split
from utils import custom_reports
import optuna
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

### Parametros conexiones

In [3]:
db_config = {
    "host": "10.43.101.189",
    "port": 3306,
    "user": "taller-mlflow",
    "password": "mlflow",
    "database": "taller",
}

tracking_uri = "http://10.43.101.189:5000"

os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://10.43.101.189:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'minioadmin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'minioadmin'

### Carga de datos desde servidor de `MySql`

In [23]:
engine = create_engine(
    f"mysql+pymysql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
)

# Crear el inspector de la base de datos
inspector = inspect(engine)

# Obtener la lista de tablas en la base de datos 'taller'
tables = inspector.get_table_names()

# Imprimir la lista de tablas
print("Tablas en la base de datos 'taller':", tables)


Tablas en la base de datos 'taller': ['iris_cleaned', 'iris_raw']


In [24]:
df = pd.read_sql_table('iris_cleaned', engine)

df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


### Set up MLflow

In [4]:
# Set up MLflow tracking URI
tracking_uri = "http://10.43.101.189:5000"
mlflow.set_tracking_uri(tracking_uri)


In [6]:
# Create an experiment
experiment_name = "iris_classification_experiment"
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='s3://mlflows3/artifacts/1', creation_time=1742159536746, experiment_id='1', last_update_time=1742159536746, lifecycle_stage='active', name='iris_classification_experiment', tags={}>

### Serparación dataset

In [26]:
X = df.drop(columns=['species'])
y = df['species']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y)

In [27]:
X_train

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
90,5.5,2.6,4.4,1.2
51,6.4,3.2,4.5,1.5
44,5.1,3.8,1.9,0.4
37,4.9,3.6,1.4,0.1
131,7.9,3.8,6.4,2.0
...,...,...,...,...
42,4.4,3.2,1.3,0.2
141,6.9,3.1,5.1,2.3
71,6.1,2.8,4.0,1.3
73,6.1,2.8,4.7,1.2


### Definición de funciones para la optimización de hiperparámetros

In [9]:
def champion_callback(study, frozen_trial):
  """
  Logging callback that will report when a new trial iteration improves upon existing
  best trial values.

  Note: This callback is not intended for use in distributed computing systems such as Spark
  or Ray due to the micro-batch iterative implementation for distributing trials to a cluster's
  workers or agents.
  The race conditions with file system state management for distributed trials will render
  inconsistent values with this callback.
  """

  winner = study.user_attrs.get("winner", None)

  if study.best_value and winner != study.best_value:
      study.set_user_attr("winner", study.best_value)
      if winner:
          improvement_percent = (abs(winner - study.best_value) / study.best_value) * 100
          print(
              f"Trial {frozen_trial.number} achieved value: {frozen_trial.value} with "
              f"{improvement_percent: .4f}% improvement"
          )
      else:
          print(f"Initial trial {frozen_trial.number} achieved value: {frozen_trial.value}")

In [10]:
# Define objective function
def objective_rf_acc(trial):
    with mlflow.start_run(nested=True):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 600), 
            'max_depth': trial.suggest_int('max_depth', 3, 30), 
            'min_samples_split': trial.suggest_int('min_samples_split', 3, 20), 
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 3, 20), 
            'max_features': trial.suggest_float('max_features', 0.5, 1.0),
            'max_samples': trial.suggest_float('max_samples', 0.7, 1.0)
        }
        
        # Initialize model
        model = RandomForestClassifier(**params, random_state=42, n_jobs=-1)

        # Define Stratified K-Folds
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

        # Compute cross-validation scores using Accuracy
        scores = cross_val_score(model, X, y, cv=skf, scoring="accuracy", n_jobs=-1)
        scores_2 = cross_val_score(model, X, y, cv=skf, scoring="roc_auc_ovo", n_jobs=-1)

        # Log parameters and metrics
        mlflow.log_params(params)
        mlflow.log_metric("mean_accuracy", np.mean(scores))
        mlflow.log_metric("mean_auroc", np.mean(scores_2))

        return np.mean(scores)

In [11]:
# Define objective function
def objective_rf_auroc(trial):
    with mlflow.start_run(nested=True):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 600), 
            'max_depth': trial.suggest_int('max_depth', 3, 30), 
            'min_samples_split': trial.suggest_int('min_samples_split', 3, 20), 
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 3, 20), 
            'max_features': trial.suggest_float('max_features', 0.5, 1.0),
            'max_samples': trial.suggest_float('max_samples', 0.7, 1.0)
        }
        
        # Initialize model
        model = RandomForestClassifier(**params, random_state=42, n_jobs=-1)

        # Define Stratified K-Folds
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

        # Compute cross-validation scores using AUROC
        scores = cross_val_score(model, X, y, cv=skf, scoring="roc_auc_ovo", n_jobs=-1)
        scores_2 = cross_val_score(model, X, y, cv=skf, scoring="accuracy", n_jobs=-1)

        # Log parameters and metrics
        mlflow.log_params(params)
        mlflow.log_metric("mean_auroc", np.mean(scores))
        mlflow.log_metric("mean_accuracy", np.mean(scores_2))

        return np.mean(scores)

### Ejecutar `runs`

In [12]:
run_name = "RandomForestClassifier optimizando Accuracy"
exp_id = mlflow.get_experiment_by_name("experiment_name")

with mlflow.start_run(experiment_id=exp_id, run_name=run_name, nested=True):

    study = optuna.create_study(direction="maximize")
    

    study.optimize(objective_rf_acc, n_trials=20, callbacks=[champion_callback])

    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_accuracy", study.best_value)

    mlflow.set_tags(
        tags={
            "project": "Taller MLFlow",
            "optimizer_engine": "optuna",
            "model_family": "RandomForestClassifier",
            "Optimization metric": "Accuracy",
        }
    )

    model = RandomForestClassifier(**study.best_params, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)
    
    test_metrics = custom_reports(model, X_test, y_test)
    mlflow.log_metrics(test_metrics)

    artifact_path = "model_rf_acc"

    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path=artifact_path,
        metadata={"model_data_version": 1},
    )

    # Get the logged model uri so that we can load it from the artifact store
    model_uri = mlflow.get_artifact_uri(artifact_path)


[I 2025-03-19 10:42:55,724] A new study created in memory with name: no-name-4f2be45c-fbc4-4eb8-9d68-1a7ef32b4c2f
[I 2025-03-19 10:43:04,455] Trial 0 finished with value: 0.945977011494253 and parameters: {'n_estimators': 303, 'max_depth': 13, 'min_samples_split': 11, 'min_samples_leaf': 16, 'max_features': 0.8408051293943295, 'max_samples': 0.9320480288077833}. Best is trial 0 with value: 0.945977011494253.


🏃 View run zealous-boar-382 at: http://10.43.101.189:5000/#/experiments/1/runs/ce11026f807149e7a6e240690905c959
🧪 View experiment at: http://10.43.101.189:5000/#/experiments/1
Initial trial 0 achieved value: 0.945977011494253


[I 2025-03-19 10:43:15,725] Trial 1 finished with value: 0.945977011494253 and parameters: {'n_estimators': 463, 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 17, 'max_features': 0.8880803633433754, 'max_samples': 0.7192275795097882}. Best is trial 0 with value: 0.945977011494253.


🏃 View run serious-lamb-411 at: http://10.43.101.189:5000/#/experiments/1/runs/3e1eabd27aa24ce78f853df076968445
🧪 View experiment at: http://10.43.101.189:5000/#/experiments/1


[I 2025-03-19 10:43:29,287] Trial 2 finished with value: 0.945977011494253 and parameters: {'n_estimators': 560, 'max_depth': 9, 'min_samples_split': 15, 'min_samples_leaf': 20, 'max_features': 0.7861954301274099, 'max_samples': 0.963221795258721}. Best is trial 0 with value: 0.945977011494253.


...

🏃 View run puzzled-carp-151 at: http://10.43.101.189:5000/#/experiments/1/runs/aaa049c908174541972052fded55c293
🧪 View experiment at: http://10.43.101.189:5000/#/experiments/1
Trial 18 achieved value: 0.9528735632183908 with  0.6996% improvement


[I 2025-03-19 10:45:53,629] Trial 19 finished with value: 0.9462068965517242 and parameters: {'n_estimators': 498, 'max_depth': 14, 'min_samples_split': 5, 'min_samples_leaf': 11, 'max_features': 0.6308898707197614, 'max_samples': 0.7422136622604663}. Best is trial 18 with value: 0.9528735632183908.


🏃 View run abrasive-ox-179 at: http://10.43.101.189:5000/#/experiments/1/runs/d784d9e4bffd4134bdd2bb86361ae92e
🧪 View experiment at: http://10.43.101.189:5000/#/experiments/1
* Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.82      0.90      0.86        10
           2       0.89      0.80      0.84        10

    accuracy                           0.90        30
   macro avg       0.90      0.90      0.90        30
weighted avg       0.90      0.90      0.90        30

* Confusion Matrix:
[[10  0  0]
 [ 0  9  1]
 [ 0  2  8]]

AUROC: 0.9900
Accuracy: 0.9000
Recall: 0.9000
Precision: 0.9024
F1 Score: 0.8997


2025/03/19 10:45:59 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/03/19 10:45:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run RandomForestClassifier optimizando Accuracy at: http://10.43.101.189:5000/#/experiments/1/runs/7390012ff4f6475c9958309fbba06852
🧪 View experiment at: http://10.43.101.189:5000/#/experiments/1


In [13]:
run_name = "RandomForestClassifier optimizando AUROC"
exp_id = mlflow.get_experiment_by_name("experiment_name")

with mlflow.start_run(experiment_id=exp_id, run_name=run_name, nested=True):

    study = optuna.create_study(direction="maximize")
    

    study.optimize(objective_rf_auroc, n_trials=20, callbacks=[champion_callback])

    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_auroc", study.best_value)

    mlflow.set_tags(
        tags={
            "project": "Taller MLFlow",
            "optimizer_engine": "optuna",
            "model_family": "RandomForestClassifier",
            "Optimization metric": "AUROC",
        }
    )

    model = RandomForestClassifier(**study.best_params, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)
    
    test_metrics = custom_reports(model, X_test, y_test)
    mlflow.log_metrics(test_metrics)

    artifact_path = "model_rf_auroc"

    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path=artifact_path,
        metadata={"model_data_version": 1},
    )

    # Get the logged model uri so that we can load it from the artifact store
    model_uri = mlflow.get_artifact_uri(artifact_path)

[I 2025-03-19 10:45:59,826] A new study created in memory with name: no-name-364e964a-c93b-4981-9b8e-a7ce5b355bd9
[I 2025-03-19 10:46:02,472] Trial 0 finished with value: 0.9957777777777779 and parameters: {'n_estimators': 116, 'max_depth': 15, 'min_samples_split': 6, 'min_samples_leaf': 16, 'max_features': 0.989216876715426, 'max_samples': 0.7817030553715953}. Best is trial 0 with value: 0.9957777777777779.


🏃 View run wistful-fowl-113 at: http://10.43.101.189:5000/#/experiments/1/runs/0cec0359e9d84ebf956dba16aba31aac
🧪 View experiment at: http://10.43.101.189:5000/#/experiments/1
Initial trial 0 achieved value: 0.9957777777777779


[I 2025-03-19 10:46:12,845] Trial 1 finished with value: 0.995037037037037 and parameters: {'n_estimators': 415, 'max_depth': 11, 'min_samples_split': 4, 'min_samples_leaf': 5, 'max_features': 0.8960616898091263, 'max_samples': 0.9053451936491401}. Best is trial 0 with value: 0.9957777777777779.


🏃 View run skittish-wasp-854 at: http://10.43.101.189:5000/#/experiments/1/runs/6072f2ac6f964e47a490962728cf02cf
🧪 View experiment at: http://10.43.101.189:5000/#/experiments/1


[I 2025-03-19 10:46:26,231] Trial 2 finished with value: 0.9965185185185185 and parameters: {'n_estimators': 558, 'max_depth': 24, 'min_samples_split': 7, 'min_samples_leaf': 16, 'max_features': 0.7901989562875102, 'max_samples': 0.7599895965171228}. Best is trial 2 with value: 0.9965185185185185.


...

🏃 View run efficient-whale-165 at: http://10.43.101.189:5000/#/experiments/1/runs/5341dcf1c6034e0a920a6f873959da81
🧪 View experiment at: http://10.43.101.189:5000/#/experiments/1


[I 2025-03-19 10:48:43,375] Trial 19 finished with value: 0.9961481481481481 and parameters: {'n_estimators': 173, 'max_depth': 18, 'min_samples_split': 7, 'min_samples_leaf': 15, 'max_features': 0.9460571999303181, 'max_samples': 0.8462717422405754}. Best is trial 2 with value: 0.9965185185185185.


🏃 View run likeable-fox-931 at: http://10.43.101.189:5000/#/experiments/1/runs/b96b093183f3432c9ecfc42d6a15c4b2
🧪 View experiment at: http://10.43.101.189:5000/#/experiments/1
* Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.82      0.90      0.86        10
           2       0.89      0.80      0.84        10

    accuracy                           0.90        30
   macro avg       0.90      0.90      0.90        30
weighted avg       0.90      0.90      0.90        30

* Confusion Matrix:
[[10  0  0]
 [ 0  9  1]
 [ 0  2  8]]

AUROC: 0.9933
Accuracy: 0.9000
Recall: 0.9000
Precision: 0.9024
F1 Score: 0.8997


2025/03/19 10:48:49 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/03/19 10:48:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run RandomForestClassifier optimizando AUROC at: http://10.43.101.189:5000/#/experiments/1/runs/e8c2eeb023744557aa3b337934d7dc68
🧪 View experiment at: http://10.43.101.189:5000/#/experiments/1


In [5]:
model_uri = "models:/iris_model@champion"
model = mlflow.pyfunc.load_model(model_uri)